In [0]:
from google.colab import drive
drive.mount("/content/drive/")

In [0]:
%cd /content/drive/My Drive/ISAG

In [0]:
!pip install git+https://github.com/qubvel/segmentation_models

In [0]:
from keras import backend as K
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from segmentation_models import Unet
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from segmentation_models.backbones import get_preprocessing


In [0]:
model_name = 'seresnext50'

model = Unet(model_name, classes=2, input_shape=(512, 512, 3), encoder_weights='imagenet')
model.summary()

In [0]:
train_x= np.load('train_x.npy')
train_y= np.load('train_y.npy')
test_x= np.load('test_x.npy')
test_y= np.load('test_y.npy')

In [0]:
train_y= np.load('train_y.npy')


In [0]:
model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])


In [0]:
y_ints = [y.argmax() for y in train_y]
classes = np.array([0,1])
class_weights = class_weight.compute_class_weight('balanced', classes, y_ints)

In [0]:
filepath="weights-improvement-{epoch:02d}-{val_iou_score:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
batch_size = 6
nb_epoch = 60

history = model.fit(train_x, train_y, 
                 batch_size=batch_size, 
                 epochs=nb_epoch, verbose=1,
                 class_weight=class_weights,
                 validation_split = 0.1,
                 callbacks=callbacks_list)

In [0]:
model.load_weights('weights-improvement-12-0.83.hdf5')

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

isim_fig = model_name + '_fig_acc.png'
plt.savefig(isim_fig)

In [0]:
test_say = 30
deneme = list(range(test_say))
for i in deneme:
    A = np.zeros((1,512,512,3))
    A[0,:,:,:] = test_x[i,:,:,:]
    B = model.predict(A)
    C = B[:,:,:,0]
    D = C>0.5
    D = D[0,:,:]
    isim = 'sonuc_' + model_name + '_' + str(i) + '.tif'
    cv2.imwrite(isim, 255*D)

In [0]:
deneme = list(range(543))
for i in deneme:
    A = test_x[i,:,:,:]   
    isim = 'test_' + str(i) + '.tif'
    cv2.imwrite(isim, A)

In [0]:
deneme = list(range(30))
for i in deneme:
    A = test_y[i,:,:,:]
    isim = 'label_' + str(i) + '.tif'
    cv2.imwrite(isim, 255*A)

In [0]:
gt = test_y[:,:,:,0]
pr = model.predict(test_x)
pr = pr[:,:,:,0]
x=pr>0.5
test_y_2 = gt.ravel()
A = x.ravel()
classification = classification_report(test_y_2, A,digits=4)
print(classification)